In [2]:
# we will only import torch and nn
import torch 
import torch.nn as nn
import math

In [58]:

# write a softmax function
def softmax(tensor , dim):
    z_max = torch.max(tensor , dim = dim , keepdim = True).values
    logits = tensor - z_max

    n = torch.exp(logits)
    sum = torch.sum(n , dim = dim , keepdim = True)

    return n / sum

In [59]:
#loss function
def crossEntropy(S , y_true):
    batch_size = S.shape[0]
    S = torch.clamp(S , 1e-12 , 1.0)
    correct_class = S[torch.arange(batch_size) , y_true]
    loss = -torch.log(correct_class)

    return loss.mean()


In [81]:
s = torch.zeros(1 , 3)
y = torch.tensor([2])
s[0 , y] = 1.0
s

tensor([[0., 0., 1.]])

In [8]:
import torch

input_dim = 3
output_dim = 2
lr = 0.1

dummy = torch.tensor([[2.0 , 3.0 , 4.0]])
y_true = torch.tensor([1])

weights = torch.randn(input_dim , output_dim , requires_grad = False)
bias = torch.zeros(output_dim)

# y = wx + b  , where x is the data
for epoch in range(50):
    z = torch.matmul(dummy , weights) + bias

    exp_z = torch.exp(z - torch.max(z)) 
    print(exp_z.shape)
    sum_ = torch.sum(exp_z, dim=-1, keepdim=True)
    print(sum_.shape)
    S = exp_z / sum_
    print(S.shape)
    loss = S[0 , y_true]
    print(loss.shape)
    loss = -torch.log(loss)

    y_hot = torch.zeros_like(S)
    y_hot[0 , y_true] = 1.0

    dz = S - y_hot

    w_ = torch.matmul(dummy.t() , dz)
    b_ = dz.sum(dim = 0)

    weights = weights - lr * w_
    bias = bias - lr * b_

    print(f"loss for epoch {epoch} | {loss.item()}")

print(f"final probs {S}")


torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 0 | 2.8380062580108643
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 1 | 0.05509262531995773
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 2 | 0.040240976959466934
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 3 | 0.03189440444111824
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 4 | 0.02649088017642498
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 5 | 0.022688308730721474
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 6 | 0.01985928975045681
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 7 | 0.01766856759786606
torch.Size([1, 2])
torch.Size([1, 1])
torch.Size([1, 2])
torch.Size([1])
loss for epoch 8 | 0.015920214

In [16]:
import torch
import torch.nn.functional as F

input_dim = 3
hidden_dim = 4  
output_dim = 2
lr = 0.1

# Data
dummy = torch.tensor([
    [2.0, 3.0, 4.0],
    [1.0, 0.5, 2.0],
    [0.0, 1.0, 0.1]
])
y_true = torch.tensor([1, 0, 1]) # Target class for each example

W1 = torch.randn(input_dim, hidden_dim, requires_grad=False)  # [3, 4]
b1 = torch.zeros(hidden_dim)  # [4]

W2 = torch.randn(hidden_dim, output_dim, requires_grad=False)  # [4, 2]
b2 = torch.zeros(output_dim)  # [2]

for epoch in range(100):
    z1 = torch.matmul(dummy , W1) + b1 # [3 , 4]
    h1 = F.relu(z1)

    z = torch.matmul(h1 , W2) + b2 # [3 , 2]

    zexp = torch.exp(z - torch.max(z)) #[3 , 2]
    sum_ = torch.sum(zexp , dim = -1 , keepdim = True) # [3 , 1]

    S = zexp / sum_ #[3 , 2]
    #

    loss = -torch.log(S[torch.arange(S.size(0)) , y_true]).mean()

    y_hot = torch.zeros_like(S) #[3 , 2]
    y_hot[torch.arange(S.size(0)) , y_true] = 1.0 #[3 , 2]

    dz2 = ( S - y_hot ) / S.size(0) # loss with respect to weight #[3 , 2]

    dw2 = torch.matmul(h1.T , dz2) # [4 , 2]
    db2 = dz2.sum(dim = 0) #[1 , 2]

    dh1 = torch.matmul(dz2 , W2.T) #[3 , 4]
    dh1[z1 <= 0] = 0 #[3 , 4]

    dw1 = torch.matmul(dummy.T , dh1) # [3 , 4]
    db1 = dh1.sum(dim = 0) #[1 , 4]

    W1 -= lr * dw1
    b1 -= lr * db1
    W2 -= lr * dw2
    b2 -= lr * db2

    print(f"loss {loss.item()}")

S

loss 2.9964773654937744
loss 0.6456148624420166
loss 0.5570704340934753
loss 0.49978017807006836
loss 0.48107874393463135
loss 0.46362969279289246
loss 0.4469797909259796
loss 0.4309196472167969
loss 0.4153747856616974
loss 0.4003027677536011
loss 0.3856752812862396
loss 0.37147197127342224
loss 0.35767948627471924
loss 0.34428858757019043
loss 0.331293523311615
loss 0.31869110465049744
loss 0.30647963285446167
loss 0.29465845227241516
loss 0.2832268178462982
loss 0.27218425273895264
loss 0.26152968406677246
loss 0.25126123428344727
loss 0.2413761168718338
loss 0.23187051713466644
loss 0.22273963689804077
loss 0.2139773964881897
loss 0.20557697117328644
loss 0.1975305825471878
loss 0.18982942402362823
loss 0.18246416747570038
loss 0.17542463541030884
loss 0.16870035231113434
loss 0.16228045523166656
loss 0.15615364909172058
loss 0.15043987333774567
loss 0.14510159194469452
loss 0.14000476896762848
loss 0.13513757288455963
loss 0.1304900050163269
loss 0.12605196237564087
loss 0.12181389

tensor([[0.0142, 0.9858],
        [0.9628, 0.0372],
        [0.0377, 0.9623]])

In [12]:
import torch

input_dim = 4
output_dim = 3
lr = 0.1

vocab_size = 5
embed_dim = 4  

E = torch.randn(vocab_size, embed_dim, requires_grad=False)

input_ids = torch.tensor([1, 3, 4])  # [3] 

y_true = torch.tensor([1])  # [1]


weights = torch.randn(embed_dim, output_dim, requires_grad=False)  # [4, 3]
bias = torch.zeros(output_dim)  # [3]


for epoch in range(50):
    x = E[input_ids]  #  [3, 4] 
    
    x_pooled = x.mean(dim=0, keepdim=True)  
    
    z = torch.matmul(x_pooled, weights) + bias  # [1, 3]
    
    # 4. Softmax
    z_max = torch.max(z, dim=-1, keepdim=True)[0]
    exp_z = torch.exp(z - z_max)
    S = exp_z / torch.sum(exp_z, dim=-1, keepdim=True)  # [1, 3]
    
    # 5. Loss
    loss = -torch.log(S[0, y_true])  # scalar
    
    # ===== BACKWARD PASS =====
    # One-hot encoding
    y_hot = torch.zeros_like(S)
    y_hot[0, y_true] = 1.0
    
    # Gradient at output
    dz = S - y_hot  # [1, 3]
    
    # Gradients for output layer
    dW = torch.matmul(x_pooled.T, dz)  # [4, 1] @ [1, 3] = [4, 3]
    db = dz.sum(dim=0)  # [3]
    
    # Gradient to pooled embedding
    dx_pooled = torch.matmul(dz, weights.T)  # [1, 3] @ [3, 4] = [1, 4]
    
    # Gradient to individual word embeddings
    # Since we averaged, gradient is distributed equally
    dx = dx_pooled.expand_as(x) / x.shape[0]  # [3, 4]
    
    # Gradient to embedding matrix
    dE = torch.zeros_like(E)
    for i, idx in enumerate(input_ids):
        dE[idx] += dx[i]  # Add gradient to each word's embedding
    
    # ===== UPDATE PARAMETERS =====
    weights -= lr * dW
    bias -= lr * db
    E -= lr * dE
    
    if epoch % 10 == 0:
        print(f"Epoch {epoch:2d} | Loss: {loss.item():.6f} | Probs: {S[0].detach().numpy()}")

print("="*50)
print(f"Final probabilities: {S[0].detach().numpy()}")
print(f"Sum: {S.sum().item():.2f}")

Epoch  0 | Loss: 0.699030 | Probs: [0.2038741  0.49706724 0.29905862]
Epoch 10 | Loss: 0.174867 | Probs: [0.07584219 0.8395685  0.08458935]
Epoch 20 | Loss: 0.087873 | Probs: [0.040824 0.915877 0.043299]
Epoch 30 | Loss: 0.056565 | Probs: [0.02697902 0.9450054  0.02801557]
Epoch 40 | Loss: 0.041005 | Probs: [0.01982892 0.95982426 0.02034688]
Final probabilities: [0.01588207 0.9679364  0.01618156]
Sum: 1.00


In [ ]:
import torch

input_dim = 4
output_dim = 3
lr = 0.1

vocab_size = 5
embed_dim = 4  

E = torch.randn(vocab_size, embed_dim, requires_grad=False) #[5 , 4]

input_ids = torch.tensor([1, 3, 4])  # [3] 

y_true = torch.tensor([1])  # [1]


weights = torch.randn(embed_dim, output_dim, requires_grad=False)  # [4, 3]
bias = torch.zeros(output_dim)  # [3]

for epoch in range(50):

    # embed
    x = E[input_ids] #  [3 , 4]
    
    #avg to handle easily
    x_ = x.mean(dim = 0 , keepdim = True)  #[1 , 4]

    #weights  
    z = torch.matmul(x_ , weights) + bias #[1 , 3] + [3] -> [1 , 3]

    z_ = torch.exp(z - torch.max(z))
    sum_ = torch.sum(z_ , dim = -1 , keepdim = True) #[1 , 1]

    S = z_ / sum_ #[1 , 3]

    loss = -torch.log(S[0 , y_true])

    y_hot = torch.zeros_like(S)
    y_hot[0 , y_true] = 1.0

    dz = S - y_hot #[1 , 3]

    dw = torch.matmul(x_.T , dz) #[4 , 3]
    db = dz.sum(dim = 0)

    dx_ = torch.matmul(dz , weights.T)
    

    dE = torch.zeros_like(E)

    for idx in input_ids:
        dE[idx] += dx_.squeeze(0)

    weights -= lr * dw
    bias -= lr * db
    E -= lr * dE
 
    print(f"Epoch {epoch:2d} | Loss: {loss.item():.6f} | Probs: {S[0].detach().numpy()}")

S

Epoch  0 | Loss: 2.518187 | Probs: [0.40348715 0.08060565 0.5159072 ]
Epoch  1 | Loss: 1.981625 | Probs: [0.3956377 0.137845  0.4665173]
Epoch  2 | Loss: 1.575094 | Probs: [0.3745425  0.20698814 0.41846943]
Epoch  3 | Loss: 1.268085 | Probs: [0.34607813 0.28137004 0.37255183]
Epoch  4 | Loss: 1.035330 | Probs: [0.31483778 0.35510927 0.33005294]
Epoch  5 | Loss: 0.857165 | Probs: [0.28383034 0.42436337 0.2918063 ]
Epoch  6 | Loss: 0.719048 | Probs: [0.25472417 0.48721588 0.25806   ]
Epoch  7 | Loss: 0.610512 | Probs: [0.22827505 0.5430727  0.22865225]
Epoch  8 | Loss: 0.524082 | Probs: [0.20469598 0.5920985  0.20320556]
Epoch  9 | Loss: 0.454394 | Probs: [0.18390748 0.63483244 0.18126003]
Epoch 10 | Loss: 0.397555 | Probs: [0.16568956 0.67196107 0.16234945]
Epoch 11 | Loss: 0.350702 | Probs: [0.14976776 0.70419335 0.14603896]
Epoch 12 | Loss: 0.311703 | Probs: [0.13585982 0.73219895 0.13194115]
Epoch 13 | Loss: 0.278946 | Probs: [0.1236991  0.75658095 0.11971997]
Epoch 14 | Loss: 0.2512

tensor([[0.0197, 0.9618, 0.0185]])